<a href="https://colab.research.google.com/github/Laura9704/Fundamentos_MCD/blob/main/Elecci%C3%B3n_Mejor_Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #gráficos
import matplotlib.dates as mdates
pd.set_option('display.max_columns', None)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math
from math import sqrt

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dropout, Dense # Se usa SimpleRNN o la capa RNN que elijas
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Importa librerías para imputación
from scipy.interpolate import interp1d

import random
import os
import warnings
warnings.filterwarnings('ignore')

# Mejor Modelo MLP

# Mejor Modelo RNN

## Simple

En este modelo el que mejor representaciones tomo fue el modelo de capas apiladas, con los siguinetes caracteriticas y parametros.  


## LTSM

## GRU

# Mejor Modelo TCN

#Eleccion Final